In [ ]:
import datetime
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain_community.llms import Ollama

with open("schema.sql", "r") as file:
    schema = file.read()

with open("menu_mappings.txt", "r") as file:
    menu_mappings = file.read()

with open("prompt_template2.txt", "r") as file:
    prompt_text = file.read()

def get_date_placeholders():
    today = datetime.date.today()
    return {
        "current_month_start": today.replace(day=1).strftime("%Y-%m-%d"),
        "current_month_end": (today.replace(day=28) + datetime.timedelta(days=4)).replace(day=1) - datetime.timedelta(days=1),
        "last_month_start": (today.replace(day=1) - datetime.timedelta(days=1)).replace(day=1).strftime("%Y-%m-%d"),
        "last_month_end": (today.replace(day=1) - datetime.timedelta(days=1)).strftime("%Y-%m-%d")
    }


llm = Ollama(
    model="llama3.2:3b",
    temperature=0,
    system="You are an SQL expert that follows rules strictly. Return ONLY valid SQL queries.",
    num_ctx=4096  # Increased context window for complex schemas
)

prompt_template = PromptTemplate(
    input_variables=["menu_mappings", "schema", "question"],
    template=prompt_text
)

sql_chain = LLMChain(llm=llm, prompt=prompt_template)


def generate_sql_query(question):
    
    dates = get_date_placeholders()
    
    raw_query = sql_chain.run({
        "menu_mappings": menu_mappings,
        "schema": schema,
        "question": question
    })
    
    final_query = raw_query.format(**dates)
    
    return final_query


question = "In which area is the biggest sales this month?"
print(generate_sql_query(question))


To find the biggest sales area this month, we need to use the `fmc.ih_sales_dd` table.

```sql
SELECT area, SUM(total_revenue) AS total_revenue
FROM fmc.ih_sales_dd
WHERE event_date BETWEEN '2025-02-01' AND '2025-02-28'
GROUP BY area
ORDER BY total_revenue DESC
LIMIT 1;
```

This query will return the area with the highest sales revenue for this month.
